# Projet simulation logicielle
* *Rhouch Oussama*
* *Cherki Inssaf*

<img src="figure/model based.png" alt="CS" style="width: 750px;"/>

## Importing libraries

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb=100"
import librosa
import librosa.display
import numpy as np
import soundfile as sf
from sklearn import preprocessing
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import zipfile
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

## Performing preprocessing on the data

##### Unzip the data

In [ ]:
zip_file_paths = ['data/data_part1.zip', 'data/data_part2.zip', 'data/data_part3.zip', 'data/data_part4.zip']

for zip_file_path in zip_file_paths:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(".")

##### Data folders

In [ ]:
noise_folder = "data/noise/"
clean_folder = "data/clean/dev-clean"
output_folder = "data/mixed"

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

### Clean audio data

##### Load clean audio data

In [ ]:
# Get the list of clean files
clean_path = []
for folder in os.listdir(clean_folder):
    folder_path = os.path.join(clean_folder, folder)
    if os.path.isdir(folder_path):
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                if file.endswith(".flac"):
                    file_path = os.path.join(root, file)
                    clean_path.append(file_path)

clean_path[-5:]

In [ ]:
# load clean data
s, s_rate = librosa.load(clean_path[0], sr=16000) 

##### plot clean audio data

In [ ]:
plt.figure(figsize=(12, 4))
librosa.display.waveshow(s, sr=s_rate)
plt.title('Clean Audio')
plt.xlabel("Time")
plt.ylabel("Amplitude")
plt.show()

##### Fourier transform of clean audio data

In [ ]:
# normalize the clean audio
s_norm = preprocessing.normalize([s])

# Fourier transform
s_stft = librosa.stft(s_norm[0])

# convert to dB
s_stft_db = librosa.amplitude_to_db(abs(s_stft))

##### plot Fourier transform of clean audio data

In [ ]:
# plot the clean audio in frequency domain
plt.figure(figsize=(14, 5))
librosa.display.specshow(s_stft_db, sr=s_rate, x_axis="time", y_axis="hz")
plt.title("Clean audio in frequency domain")
plt.xlabel("Time")
plt.ylabel("Frequency")
plt.colorbar()
plt.show()

### Noise audio data

##### Load noise audio data

In [ ]:
# Get the list of noise files
noise_file = ""
if os.path.isdir(noise_folder):
        for root, dirs, files in os.walk(noise_folder):
            for file in files:
                if file.endswith(".wav"):
                    noise_file = os.path.join(root, file)

noise_file

In [ ]:
# load noise data
u, u_rate = librosa.load(noise_file, sr=16000)

##### plot noise audio data

In [ ]:
# Plot the noise audio
plt.figure(figsize=(14, 5))
librosa.display.waveshow(u, sr=u_rate)
plt.title("Noise audio")
plt.xlabel("Time")
plt.ylabel("Amplitude")
plt.show()

##### Fourier transform of noise audio data

In [ ]:
# normalize the clean audio
u_norm = preprocessing.normalize([u[:len(s)]]) 

# Fourier transform
u_stft = librosa.stft(u_norm[0])

# convert to dB
u_stft_db = librosa.amplitude_to_db(abs(u_stft))

##### plot Fourier transform of noise audio data

In [ ]:
# plot the noise audio in frequency domain
plt.figure(figsize=(14, 5))
librosa.display.specshow(u_stft_db, sr=u_rate, x_axis="time", y_axis="hz")
plt.title("Noise audio in frequency domain")
plt.xlabel("Time")
plt.ylabel("Frequency")
plt.colorbar()
plt.show()

### Noisy audio data

##### RSB: ratio signal to noise

In [ ]:
# RSB
rsb = 10 * np.log10(np.sum(s_stft**2)/np.sum(u_stft**2))
rsb

##### alpha: noise factor

In [ ]:
# alpha
alpha = 10**(-rsb/20)
alpha

##### Noisy audio data

In [ ]:
# mix the clean and noise audio
x = s_norm + alpha * u_norm
x = x.astype(float)

x.shape

##### plot noisy audio data

In [ ]:
# plot the noisy audio
plt.figure(figsize=(14, 5))
librosa.display.waveshow(x, sr=s_rate)
plt.title("Mixed audio")
plt.xlabel("Time")
plt.ylabel("Amplitude")
plt.show()

##### Fourier transform of noisy audio data

In [ ]:
# Fourier transform of the noisy audio
x_stft = librosa.stft(x[0])

# convert to dB
x_stft_db = librosa.amplitude_to_db(abs(x_stft))

##### plot Fourier transform of noisy audio data

In [ ]:
# plot the noisy audio in frequency domain
plt.figure(figsize=(14, 5))
librosa.display.specshow(x_stft_db, sr=s_rate, x_axis="time", y_axis="hz")
plt.title("Mixed audio in frequency domain")
plt.xlabel("Time")
plt.ylabel("Frequency")
plt.colorbar()
plt.show()

In [ ]:
U, _ = librosa.magphase(u_stft)
rms = librosa.feature.rms(S=U)

In [ ]:
fig, ax = plt.subplots(figsize=(14, 5), nrows=2, sharex=True)
times = librosa.times_like(rms)
ax[0].semilogy(times, rms[0], label='RMS Energy')
ax[0].set(xticks=[], xlim=[times.min(), times.max()], ylabel='RMS Energy')
ax[0].legend()
ax[0].label_outer()
librosa.display.specshow(librosa.amplitude_to_db(U), y_axis='log', x_axis='time', ax=ax[1])
ax[1].set(title='log Power spectrogram')
plt.show()

In [ ]:
s_list = []
s_rate_list = []
for clean_data in clean_path:
    s, s_rate = librosa.load(clean_path[0], sr=16000)
    s_list.append(s)
    s_rate_list.append(s_rate)

In [ ]:
class Preprocessing(Dataset):
    def __init__(self, clean_path, noise_file):
        self.clean_files = []
        self.noisy_files = []
        self.clean_path = clean_path
        self.noise_file = noise_file
        self.prepare_data()
        self.adjust_length()

    def mix(self, clean_audio_path, noise_file):
        s, _ = librosa.load(clean_audio_path, sr=16000)
        u_audio, _ = librosa.load(noise_file, sr=16000)

        # Ensure the clean and noisy audio have the same length
        min_length = min(len(s), len(u_audio))
        s = s[:min_length]
        u_audio = u_audio[:min_length]

        s_norm = preprocessing.normalize([s])
        u_norm = preprocessing.normalize([u_audio])
        
        s_stft = librosa.stft(s_norm[0])
        u_stft = librosa.stft(u_norm[0])

        rsb = 10 * np.log10(np.sum(s_stft**2) / np.sum(u_stft**2))
        alpha = 10**(-rsb / 20)
        x = s_norm + alpha * u_norm
        
        s_norm = torch.tensor(np.transpose(np.abs(librosa.stft(s_norm, n_fft=1024, hop_length=512)))).cuda()
        x = torch.tensor(np.transpose(np.abs(librosa.stft(x, n_fft=1024, hop_length=512)))).cuda()

        return s_norm, x

    def prepare_data(self):
        for clean_audio_path in self.clean_path:
            clean, noisy = self.mix(clean_audio_path, self.noise_file)
            self.clean_files.append(clean)
            self.noisy_files.append(noisy)
    
    #adjust the length of the audio files
    def adjust_length(self):
        # Get the minimum shape of the clean audio files
        pad_shape = min([file.shape[0] for file in self.clean_files])
        for i in range(len(self.clean_files)):
            diff_pad = pad_shape - self.clean_files[i].shape[0]
            self.clean_files[i] = F.pad(self.clean_files[i], (0, 0, diff_pad, 0))
            self.noisy_files[i] = F.pad(self.noisy_files[i], (0, 0, diff_pad, 0))

    def __len__(self):
        return len(self.clean_files)

    def __getitem__(self, idx):
        return (torch.tensor(self.noisy_files[idx]), torch.tensor(self.clean_files[idx]))


In [ ]:
# Split data into training and test sets (80% training, 20% test)
clean_train, clean_test = train_test_split(clean_path[:20], test_size=0.2, random_state=42)

In [ ]:
# Create a Preprocessing object for the training set
preprocessor_train = Preprocessing(clean_train, noise_file)

# Create a Preprocessing object for the test set
preprocessor_test = Preprocessing(clean_test, noise_file)

In [ ]:
len(preprocessor_train), len(preprocessor_test)

In [ ]:
train_loader = DataLoader(dataset=preprocessor_train, batch_size=100, shuffle=True)
test_loader = DataLoader(dataset=preprocessor_test, batch_size=100, shuffle=False)

In [ ]:
# check if train_loader have tensors of equal size
for noisy, clean in train_loader:
    print(noisy.shape)
    print(clean.shape)
    break

In [ ]:
class network_1D(nn.Module):
    def __init__(self):
        super(network_1D, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=9, stride=1, padding=4)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=9, stride=1, padding=4)
        self.conv3 = nn.Conv1d(in_channels=32, out_channels=1, kernel_size=9, stride=1, padding=4)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.sigmoid(self.conv3(x))
        return x

In [ ]:
class network_2D(nn.Module):
    def __init__(self):
        super(network_2D, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(9, 9), stride=1, padding=(4, 4))
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(9, 9), stride=1, padding=(4, 4))
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=1, kernel_size=(9, 9), stride=1, padding=(4, 4))
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.sigmoid(self.conv3(x))
        return x

In [ ]:
class network_RNN(nn.Module):
    def __init__(self):
        super(network_RNN, self).__init__()
        self.rnn = nn.RNN(input_size=1, hidden_size=64, num_layers=2, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(128, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x, _ = self.rnn(x)
        x = self.sigmoid(self.fc(x))
        return x

In [ ]:
#train each model
def train(model, train_loader, optimizer, criterion, epoch):
    model.train()
    for batch_idx, (noisy, clean) in enumerate(train_loader):
        # Find the maximum size in the current batch
        max_size = max(noisy.size(1), clean.size(1))
        
        # Pad the tensors to the maximum size
        noisy = F.pad(noisy.unsqueeze(1), (0, max_size - noisy.size(1)))
        clean = F.pad(clean.unsqueeze(1), (0, max_size - clean.size(1)))
        
        optimizer.zero_grad()
        output = model(noisy)
        loss = criterion(output, clean)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\t Average loss: {:.6f}'.format(
                epoch, batch_idx * len(noisy), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
def test(model, test_loader, criterion):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for noisy, clean in test_loader:
            noisy = noisy.unsqueeze(1)
            clean = clean.unsqueeze(1)
            output = model(noisy)
            test_loss += criterion(output, clean).item()
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.6f}'.format(test_loss))
    return test_loss

def save_model(model, epoch, optimizer, loss, path):
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss
    }, path)
    
def load_model(model, optimizer, path):
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    return model, optimizer, epoch, loss

In [ ]:
# now train each model with data
# 1D CNN
model_1D = network_1D()
optimizer_1D = optim.Adam(model_1D.parameters(), lr=0.001)
criterion = nn.MSELoss()
n_epochs = 10
path = "model_1D.pth"
train_loss_1D = []
test_loss_1D = []
for epoch in range(1, n_epochs + 1):
    train(model_1D, train_loader, optimizer_1D, criterion, epoch)
    train_loss_1D.append(test(model_1D, train_loader, criterion))
    test_loss_1D.append(test(model_1D, test_loader, criterion))
    save_model(model_1D, epoch, optimizer_1D, test_loss_1D[-1], path)


In [ ]:
#2D CNN
